In [1]:
from IPython import __version__ as ipython_version
from pandas import __version__ as pandas_version
from bokeh import __version__ as bokeh_version
print(f'IPython - {ipython_version}')
print(f'Pandas - {pandas_version}')
print(f'Bokeh - {bokeh_version}')

IPython - 6.3.1
Pandas - 0.22.0
Bokeh - 0.12.14


In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, date

In [3]:
import local_db
connection = local_db.connection()

In [4]:
today = datetime.now().strftime('%Y%m%d')
print(today)

20180427


In [5]:
begin_year = '2009'

<H2>Stage Ranking codes</H2>

In [6]:
sql_str = "SELECT STAGERANKING_ID, field_name, field_value " + \
          "FROM VWSSTAGERANKING WHERE " + \
          "status = 'A' "
stgrnk = pd.read_sql_query(sql_str, connection)

In [ ]:
# read VWSTAGERANKING.csv and drop unused fields
#stgrnk = pd.read_csv('VWSTAGERANKING.csv')
#stgrnk.drop(['code_table', 'MEDIUM_DESC',
#             'Converted/Confirmed/Accepted/Require SepDate'],
#            inplace=True, axis=1)

In [ ]:
print('stgrnk', stgrnk.shape)

In [ ]:
print('stgrnk\n', stgrnk.dtypes)

In [ ]:
print(stgrnk.columns)

<H2>Stage History data</H2>

In [7]:
sql_str = "SELECT PEOPLE_CODE_ID, ACADEMIC_YEAR, ACADEMIC_TERM, " + \
          "ACADEMIC_SESSION, FIELD_ID, FIELD_DATE " + \
          "FROM STAGEHISTORY WHERE " + \
          f"ACADEMIC_YEAR >= '{begin_year}' "
stg_hist = pd.read_sql_query(sql_str, connection)

In [ ]:
# read STAGEHISTORY.csv
#stg_hist_dtype = {'PEOPLE_CODE_ID': str, 'ACADEMIC_YEAR': str,
#                  'ACADEMIC_TERM': str, 'ACADEMIC_SESSION': str,
#                  'FIELD_ID': np.int64}
#date_cols = ['FIELD_DATE']
#stg_hist = pd.read_csv('STAGEHISTORY.csv', dtype=stg_hist_dtype,
#                       parse_dates=date_cols,
#                       usecols=['PEOPLE_CODE_ID', 'ACADEMIC_YEAR',
#                                'ACADEMIC_TERM', 'ACADEMIC_SESSION',
#                                'FIELD_ID', 'FIELD_DATE'])

In [8]:
print('stg_hist', stg_hist.shape)

stg_hist (158213, 6)


In [ ]:
print('stg_hist')
print(stg_hist.dtypes)

In [9]:
stg_hist = stg_hist.rename(columns={'FIELD_DATE': 'create_date'})

In [10]:
stage_data = pd.merge(stg_hist, stgrnk, left_on=['FIELD_ID'],
                      right_on=['STAGERANKING_ID'], how='left')

In [11]:
keep_fields = ['PEOPLE_CODE_ID', 'ACADEMIC_YEAR', 'ACADEMIC_TERM',
               'ACADEMIC_SESSION', 'field_name', 'field_value', 'create_date']
stage_data = stage_data.loc[~stage_data['create_date'].isnull(), keep_fields]

In [12]:
print('stage_data', stage_data.shape)

stage_data (158213, 7)


In [ ]:
print('stage_data')
print(stage_data.dtypes)
stage_data.head()

<H2>Academic Data</H2>

In [13]:
sql_str = "SELECT PEOPLE_CODE_ID, ACADEMIC_YEAR, ACADEMIC_TERM, " + \
          "ACADEMIC_SESSION, POPULATION, INQUIRY_FLAG, " + \
          "APPLICATION_FLAG, APPLICATION_DATE, " + \
          "APP_STATUS, APP_STATUS_DATE, " +\
          "APP_DECISION, APP_DECISION_DATE " + \
          "FROM ACADEMIC WHERE " + \
          f"ACADEMIC_YEAR >= '{begin_year}' "
academic = pd.read_sql_query(sql_str, connection)

In [ ]:
academic_dtype = {'PEOPLE_CODE_ID': str, 'ACADEMIC_YEAR': str,
                  'ACADEMIC_TERM': str, 'ACADEMIC_SESSION': str,
                  'APPLICATION_FLAG': str, 'APP_STATUS': str}
date_cols = ['APPLICATION_DATE', 'APP_STATUS_DATE', 'APP_DECISION_DATE']
academic = pd.read_csv('ACADEMIC.csv', dtype=academic_dtype,
                       parse_dates=date_cols,
                       usecols=['PEOPLE_CODE_ID',
                                'ACADEMIC_YEAR', 'ACADEMIC_TERM',
                                'ACADEMIC_SESSION', 'POPULATION',
                                'INQUIRY_FLAG',
                                'APPLICATION_FLAG', 'APPLICATION_DATE',
                                'APP_STATUS', 'APP_STATUS_DATE',
                                'APP_DECISION', 'APP_DECISION_DATE'])

In [14]:
print('academic', academic.shape)
print('academic')
print(academic.dtypes)
academic.info()

academic (84746, 12)
academic
PEOPLE_CODE_ID               object
ACADEMIC_YEAR                object
ACADEMIC_TERM                object
ACADEMIC_SESSION             object
POPULATION                   object
INQUIRY_FLAG                 object
APPLICATION_FLAG             object
APPLICATION_DATE             object
APP_STATUS                   object
APP_STATUS_DATE      datetime64[ns]
APP_DECISION                 object
APP_DECISION_DATE    datetime64[ns]
dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84746 entries, 0 to 84745
Data columns (total 12 columns):
PEOPLE_CODE_ID       84746 non-null object
ACADEMIC_YEAR        84746 non-null object
ACADEMIC_TERM        84746 non-null object
ACADEMIC_SESSION     84746 non-null object
POPULATION           84746 non-null object
INQUIRY_FLAG         84746 non-null object
APPLICATION_FLAG     84746 non-null object
APPLICATION_DATE     13663 non-null object
APP_STATUS           13601 non-null object
APP_STATUS_DATE      13598 n

In [ ]:
print(academic['POPULATION'].value_counts().sort_index())
print()
print(academic['POPULATION'].value_counts().sum())

In [15]:
app_data = (academic.loc[~(academic['POPULATION'].isin(['ADVSTU', 'NOND'])) &
                         ((academic['INQUIRY_FLAG'] == 'Y') |
                          (academic['APPLICATION_FLAG'] == 'Y'))]
            )

print('app_data', app_data.shape)
print('app_data')
print(app_data.dtypes)

app_data (32422, 12)
app_data
PEOPLE_CODE_ID               object
ACADEMIC_YEAR                object
ACADEMIC_TERM                object
ACADEMIC_SESSION             object
POPULATION                   object
INQUIRY_FLAG                 object
APPLICATION_FLAG             object
APPLICATION_DATE             object
APP_STATUS                   object
APP_STATUS_DATE      datetime64[ns]
APP_DECISION                 object
APP_DECISION_DATE    datetime64[ns]
dtype: object


In [16]:
app_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32422 entries, 26 to 84745
Data columns (total 12 columns):
PEOPLE_CODE_ID       32422 non-null object
ACADEMIC_YEAR        32422 non-null object
ACADEMIC_TERM        32422 non-null object
ACADEMIC_SESSION     32422 non-null object
POPULATION           32422 non-null object
INQUIRY_FLAG         32422 non-null object
APPLICATION_FLAG     32422 non-null object
APPLICATION_DATE     13659 non-null object
APP_STATUS           13597 non-null object
APP_STATUS_DATE      13594 non-null datetime64[ns]
APP_DECISION         13588 non-null object
APP_DECISION_DATE    13584 non-null datetime64[ns]
dtypes: datetime64[ns](2), object(10)
memory usage: 3.2+ MB


In [17]:
applied = (app_data[app_data['APP_STATUS'].notnull()]
           .rename(columns={'APP_STATUS': 'field_value'})
           .rename(columns={'APP_STATUS_DATE': 'create_date'})
           )
applied.loc[:, 'field_name'] = 'Application Status'
applied = applied.loc[~applied['create_date'].isnull(), keep_fields]

print('applied', applied.shape)
print('applied')
print(applied.dtypes)

applied (13594, 7)
applied
PEOPLE_CODE_ID              object
ACADEMIC_YEAR               object
ACADEMIC_TERM               object
ACADEMIC_SESSION            object
field_name                  object
field_value                 object
create_date         datetime64[ns]
dtype: object


In [ ]:
print(applied['field_value'].value_counts().sort_index())
print(applied['field_value'].value_counts().sum())

In [ ]:
applied.info()

In [18]:
accepted = (app_data[app_data['APP_DECISION'].notnull()]
            .rename(columns={'APP_DECISION': 'field_value'})
            .rename(columns={'APP_DECISION_DATE': 'create_date'})
            )
accepted.loc[:, 'field_name'] = 'Application Decision'
accepted = accepted.loc[~accepted['create_date'].isnull(), keep_fields]


print('accepted', accepted.shape)
print('accepted')
print(accepted.dtypes)

accepted (13584, 7)
accepted
PEOPLE_CODE_ID              object
ACADEMIC_YEAR               object
ACADEMIC_TERM               object
ACADEMIC_SESSION            object
field_name                  object
field_value                 object
create_date         datetime64[ns]
dtype: object


In [ ]:
print(accepted['field_value'].value_counts().sort_index())
print(accepted['field_value'].value_counts().sum())

In [ ]:
accepted.info()

In [19]:
# stack Stage History, Academic Applied and Academic Accepted
adm_df = stage_data.append(applied).append(accepted)

print('adm_df', adm_df.shape)
print('adm_df')
print(adm_df.dtypes)

adm_df (185391, 7)
adm_df
PEOPLE_CODE_ID              object
ACADEMIC_YEAR               object
ACADEMIC_TERM               object
ACADEMIC_SESSION            object
field_name                  object
field_value                 object
create_date         datetime64[ns]
dtype: object


In [ ]:
adm_df.head()

In [ ]:
adm_df.info()

In [20]:
adm_df = (adm_df.loc[((adm_df['ACADEMIC_TERM'].isin(['FALL', 'SPRING'])) &
                      (adm_df['ACADEMIC_SESSION'] == 'MAIN') &
                      (adm_df['ACADEMIC_YEAR'] >= '2009')
                      )
                     ]
          )

print('adm_df', adm_df.shape)
print('adm_df')
print(adm_df.dtypes)

adm_df (123286, 7)
adm_df
PEOPLE_CODE_ID              object
ACADEMIC_YEAR               object
ACADEMIC_TERM               object
ACADEMIC_SESSION            object
field_name                  object
field_value                 object
create_date         datetime64[ns]
dtype: object


In [21]:
# create new fields
adm_df['year_term'] = (adm_df['ACADEMIC_YEAR'] + '.' +
                       adm_df['ACADEMIC_TERM'].str.title())
adm_df['Week_Number'] = adm_df['create_date'].dt.week

In [22]:
# convert ACADEMIC_YEAR to numeric keep numeric-valued records
adm_df['ACADEMIC_YEAR'] = pd.to_numeric(adm_df['ACADEMIC_YEAR'],
                                        errors='coerce', downcast='integer')
adm_df = adm_df.loc[adm_df['ACADEMIC_YEAR'].notnull()]

In [23]:
adm_week_number = (lambda r: (r['Week_Number'] -
                              (date(int(r['ACADEMIC_YEAR']), 9, 1)
                              .isocalendar()[1])
                              )
                   if (r['Week_Number'] > (date(int(r['ACADEMIC_YEAR']), 9, 1)
                                           .isocalendar()[1]))
                   else (53 + r['Week_Number'] -
                         (date(int(r['ACADEMIC_YEAR']), 9, 1)
                         .isocalendar()[1])
                         )
                   )


In [24]:
adm_df['Admissions_Week'] = adm_df.apply(adm_week_number, axis=1)

print('adm_df', adm_df.shape)
print('adm_df')
print(adm_df.dtypes)

adm_df (123286, 10)
adm_df
PEOPLE_CODE_ID              object
ACADEMIC_YEAR                int16
ACADEMIC_TERM               object
ACADEMIC_SESSION            object
field_name                  object
field_value                 object
create_date         datetime64[ns]
year_term                   object
Week_Number                  int64
Admissions_Week              int64
dtype: object


In [ ]:
adm_df.info()

In [ ]:
adm_df.head()

In [ ]:
adm_df.columns

In [ ]:
adm_df[(adm_df['year_term']=='2014.Fall') & (adm_df['PEOPLE_CODE_ID']=='P000026232')].head(30)

In [26]:
print(adm_df['field_value'].value_counts().sort_index())
print(adm_df['field_value'].value_counts().sum())

300     10103
500      7602
ACC     12927
ACXL     3430
CANC     6786
COMP     4139
DEF         4
DEFR      399
DENY       91
DPAC     4605
PEND     7166
TRDP      662
TRNS      808
TRPD        4
Name: field_value, dtype: int64
58726


In [25]:
adm_keep_values = ['300', 'ACC', 'ACXL', 'CANC', 'DEF', 'DEFR', 'DENY', 'DPAC',
                   'TRDP', 'TRPD', 'TRNS', 'WAIT', '500', 'PEND', 'COMP']
adm_keep_cols = ['PEOPLE_CODE_ID', 'year_term', 'Admissions_Week',
                 'field_value']
adm_df = adm_df.loc[(adm_df['field_value'].isin(adm_keep_values)),
                    adm_keep_cols]

print('adm_df', adm_df.shape)
print('adm_df')
print(adm_df.dtypes)

adm_df (58726, 4)
adm_df
PEOPLE_CODE_ID     object
year_term          object
Admissions_Week     int64
field_value        object
dtype: object


In [ ]:
adm_df.head()

In [ ]:
adm_df[(adm_df['year_term']=='2014.Fall') & (adm_df['PEOPLE_CODE_ID']=='P000026232')].head(30)

In [27]:
# admissions status table
admission_status = {'300': 'Applied', 'ACC': 'Accepted', 'ACXL': 'Canceled',
                    'CANC': 'Canceled', 'DEF': 'Canceled', 'DEFR': 'Canceled',
                    'DENY': 'Canceled', 'DPAC': 'Deposited',
                    'TRDP': 'Deposited', 'TRPD': 'Deposited',
                    'TRNS': 'Accepted', 'WAIT': 'Accepted',
                    '500': 'Deposited', 'PEND': 'Applied',
                    'COMP': 'Applied',
                    }
adm_stat = pd.DataFrame(list(admission_status.items()),
                        columns=['field_value', 'admission_status'])

In [76]:
adm_df1 = (pd.merge(adm_df, adm_stat, on=['field_value'], how='left')
           .drop(['field_value'], axis=1)
           .drop_duplicates(['PEOPLE_CODE_ID', 'year_term', 'Admissions_Week',
                             'admission_status'])
           )

print('adm_df1', adm_df1.shape)
print('adm_df1')
print(adm_df1.dtypes)

adm_df1 (29725, 4)
adm_df1
PEOPLE_CODE_ID      object
year_term           object
Admissions_Week      int64
admission_status    object
dtype: object


In [ ]:
adm_df1.head()

In [ ]:
adm_df1[(adm_df1['year_term']=='2014.Fall') & (adm_df1['admission_status']=='Deposited')].head(30)

In [ ]:
adm_df1[(adm_df1['year_term']=='2014.Fall') & (adm_df1['PEOPLE_CODE_ID']=='P000026232')].head(30)

In [77]:
adm_df1 = (adm_df1.sort_values(['year_term', 'PEOPLE_CODE_ID',
                                'admission_status', 'Admissions_Week'])
           .drop_duplicates(['year_term', 'PEOPLE_CODE_ID',
                             'admission_status'],
                            keep='first')
           )

In [78]:
print('adm_df1', adm_df1.shape)

adm_df1 (26088, 4)


In [ ]:
adm_df1[(adm_df1['year_term']=='2014.Fall') & (adm_df1['admission_status']=='Deposited')].head(30)

In [ ]:
adm_df1[(adm_df1['year_term']=='2014.Fall') & (adm_df1['PEOPLE_CODE_ID']=='P000026232')].head(30)

In [ ]:
adm_df2 = (adm_df1.loc[(adm_df1['year_term'].isin(['2014.Fall', '2015.Fall']))]
           .reset_index()
           .set_index(['year_term', 'PEOPLE_CODE_ID', 'admission_status'])
           .drop(['index'], axis=1)
           .unstack(level=-1)
          )

print('adm_df2', adm_df2.shape)
print('adm_df2')
print(adm_df2.dtypes)

In [ ]:
adm_df2.head()

In [ ]:
#q = adm_df1.loc[(adm_df1['year_term']=='2014.Fall'), ['PEOPLE_CODE_ID', 'Admissions_Week', 'admission_status']]
q = adm_df1.loc[((adm_df1['year_term']=='2014.Fall') | (adm_df1['year_term']=='2015.Fall')), ['year_term', 'PEOPLE_CODE_ID', 'Admissions_Week', 'admission_status']]

In [ ]:
print(q.shape)
q.head()

In [ ]:
w = q.set_index(['year_term', 'PEOPLE_CODE_ID'])

In [ ]:
print(w.shape)
w.head(30)

In [32]:
# just 2018.Fall for testing
adm_df1 = adm_df1.loc[(adm_df1['year_term']=='2018.Fall') , ['year_term', 'PEOPLE_CODE_ID', 'Admissions_Week', 'admission_status']]

In [79]:
e = adm_df1.pivot_table(index=['year_term', 'PEOPLE_CODE_ID'],
                        columns=['admission_status'],
                        values=['Admissions_Week']
                        )
e = e.fillna(54)
print(e.shape)
e.head(30)

(11523, 4)


Admissions_Week                           
admission_status                Accepted Applied Canceled Deposited
year_term PEOPLE_CODE_ID                                           
2009.Fall P000000564                25.0    54.0     54.0      25.0
          P000006539                37.0    54.0     54.0      37.0
          P000022053                29.0    54.0     54.0      29.0
          P000022130                28.0    54.0     54.0      28.0
          P000022259                26.0    54.0     54.0      26.0
          P000022701                50.0    50.0     54.0      50.0
          P000022722                 3.0     3.0     54.0      43.0
          P000022725                54.0    49.0     49.0      54.0
          P000022727                 5.0    54.0     54.0       5.0
          P000022729                 6.0    54.0     54.0       6.0
          P000022730                 6.0    54.0     54.0       6.0
          P000022734                 7.0    54.0     54.0       7.0
          P000022736                 7.0    54.0     54.0       7.0
          P000022758                 9.0    54.0     54.0       9.0
          P000022759                 9.0    54.0     54.0       9.0
          P000022761                 9.0    54.0     54.0       9.0
          P000022762                 9.0    54.0     54.0       9.0
          P000022768                10.0    54.0     54.0      10.0
          P000022769                10.0    54.0     54.0      10.0
          P000022774                11.0    54.0     41.0      11.0
          P000022776                11.0    54.0     54.0      11.0
          P000022782                11.0    54.0     54.0      11.0
          P000022783                11.0    54.0     54.0      11.0
          P000022785                11.0    54.0     54.0      11.0
          P000022787                12.0    54.0     54.0      12.0
          P000022788                12.0    54.0     54.0      12.0
          P000022796                13.0    54.0     54.0      13.0
          P000022797                13.0    54.0     54.0      13.0
          P000022798                13.0    54.0     54.0      13.0
          P000022799                13.0    54.0     54.0      13.0

In [54]:
e.to_csv('2018_Fall_pivot.csv')

In [80]:
# function returns status for week
def f_status(field, data_frame, n):
    f_week = (lambda df: 1
              if ((df[('Admissions_Week', field)] <= n) &
                  (df[('Admissions_Week', 'Canceled')] > n))
              else 0
              )
    return data_frame.apply(f_week, axis=1)

In [56]:
e[('Admissions_Week', 'Canceled')]

year_term  PEOPLE_CODE_ID
2018.Fall  P000026021        54.0
           P000037722        54.0
           P000041297        54.0
           P000044821        54.0
           P000045054        54.0
           P000045209        54.0
           P000045865        54.0
           P000047104        54.0
           P000047242        54.0
           P000050259        54.0
           P000050922        54.0
           P000051053        54.0
           P000051145        54.0
           P000051235        54.0
           P000051293        54.0
           P000051681        54.0
           P000051745        54.0
           P000052188        54.0
           P000053208        54.0
           P000053326        54.0
           P000053398        54.0
           P000053572        54.0
           P000053576        54.0
           P000053577        54.0
           P000053580        54.0
           P000053595        54.0
           P000053870        54.0
           P000053874        54.0
           P000053942 

In [57]:
f_status('Applied', e, 1).to_csv('2018_Fall_w1.csv')

In [81]:
# function returns DataFrame of 53 week status values
def fill_weeks(field, data_frame):
    weeks = range(1, 54)
    r = pd.DataFrame(np.zeros((data_frame.shape[0], 53)),
                     index=data_frame.index,
                     columns=[f'{w:02d}' for w in weeks])
    for w in weeks:
        f = f'{w:02d}'
        r.loc[:, f] = f_status(field, data_frame, w)
        r.loc[:, 'stage'] = field

    r = r.reset_index().set_index(['year_term', 'stage', 'PEOPLE_CODE_ID'])
   
    return r

In [82]:
stage_list = ['Applied', 'Accepted', 'Deposited']
w = pd.DataFrame()
for stg in stage_list:
    w = pd.concat([w, fill_weeks(stg, e)])


In [69]:
w.to_csv('2018_Fall_stages.csv')

In [70]:
w.head()


01  02  03  04  05  06  07  08  09  10 ...  \
year_term stage   PEOPLE_CODE_ID                                         ...   
2018.Fall Applied P000026021       0   0   0   0   0   0   0   0   0   0 ...   
                  P000037722       0   0   0   0   0   0   0   0   0   0 ...   
                  P000041297       0   0   0   0   0   0   0   0   0   0 ...   
                  P000044821       0   0   0   0   0   0   0   0   0   0 ...   
                  P000045054       0   0   0   0   0   0   0   0   0   0 ...   

                                  44  45  46  47  48  49  50  51  52  53  
year_term stage   PEOPLE_CODE_ID                                          
2018.Fall Applied P000026021       1   1   1   1   1   1   1   1   1   1  
                  P000037722       1   1   1   1   1   1   1   1   1   1  
                  P000041297       1   1   1   1   1   1   1   1   1   1  
                  P000044821       1   1   1   1   1   1   1   1   1   1  
                  P000045054       1   1   1   1   1   1   1   1   1   1  

[5 rows x 53 columns]

In [71]:

print(w.shape)
w.head(30)

(2487, 53)


01  02  03  04  05  06  07  08  09  10 ...  \
year_term stage   PEOPLE_CODE_ID                                         ...   
2018.Fall Applied P000026021       0   0   0   0   0   0   0   0   0   0 ...   
                  P000037722       0   0   0   0   0   0   0   0   0   0 ...   
                  P000041297       0   0   0   0   0   0   0   0   0   0 ...   
                  P000044821       0   0   0   0   0   0   0   0   0   0 ...   
                  P000045054       0   0   0   0   0   0   0   0   0   0 ...   
                  P000045209       0   0   0   0   0   1   1   1   1   1 ...   
                  P000045865       0   0   0   0   0   0   0   0   0   0 ...   
                  P000047104       0   0   0   0   0   0   0   0   0   0 ...   
                  P000047242       0   0   0   0   0   0   0   0   1   1 ...   
                  P000050259       0   0   0   0   0   0   0   0   0   0 ...   
                  P000050922       0   0   0   0   0   0   0   0   0   0 ...   
                  P000051053       0   0   0   0   0   0   0   0   0   0 ...   
                  P000051145       0   0   0   0   0   0   0   0   1   1 ...   
                  P000051235       0   0   0   0   0   0   0   0   0   0 ...   
                  P000051293       0   0   0   0   0   1   1   1   1   1 ...   
                  P000051681       0   0   0   0   0   0   0   0   0   0 ...   
                  P000051745       0   0   0   0   0   0   1   1   1   1 ...   
                  P000052188       0   0   0   0   0   0   0   0   0   0 ...   
                  P000053208       0   0   0   0   0   0   0   0   0   0 ...   
                  P000053326       0   0   0   0   0   0   0   0   0   0 ...   
                  P000053398       0   0   0   1   1   1   1   1   1   1 ...   
                  P000053572       0   0   0   0   0   0   0   0   0   0 ...   
                  P000053576       0   0   0   0   0   0   0   1   1   1 ...   
                  P000053577       0   0   0   0   0   0   1   1   1   1 ...   
                  P000053580       0   0   0   0   0   0   0   1   1   1 ...   
                  P000053595       0   0   0   0   0   0   0   0   0   0 ...   
                  P000053870       0   0   0   0   0   0   0   0   0   0 ...   
                  P000053874       0   0   0   0   0   0   0   0   0   0 ...   
                  P000053942       0   0   0   0   0   0   0   0   1   1 ...   
                  P000053976       0   0   0   0   0   0   0   0   0   0 ...   

                                  44  45  46  47  48  49  50  51  52  53  
year_term stage   PEOPLE_CODE_ID                                          
2018.Fall Applied P000026021       1   1   1   1   1   1   1   1   1   1  
                  P000037722       1   1   1   1   1   1   1   1   1   1  
                  P000041297       1   1   1   1   1   1   1   1   1   1  
                  P000044821       1   1   1   1   1   1   1   1   1   1  
                  P000045054       1   1   1   1   1   1   1   1   1   1  
                  P000045209       1   1   1   1   1   1   1   1   1   1  
                  P000045865       1   1   1   1   1   1   1   1   1   1  
                  P000047104       1   1   1   1   1   1   1   1   1   1  
                  P000047242       1   1   1   1   1   1   1   1   1   1  
                  P000050259       1   1   1   1   1   1   1   1   1   1  
                  P000050922       1   1   1   1   1   1   1   1   1   1  
                  P000051053       1   1   1   1   1   1   1   1   1   1  
                  P000051145       1   1   1   1   1   1   1   1   1   1  
                  P000051235       1   1   1   1   1   1   1   1   1   1  
                  P000051293       1   1   1   1   1   1   1   1   1   1  
                  P000051681       1   1   1   1   1   1   1   1   1   1  
                  P000051745       1   1   1   1   1   1   1   1   1   1  
                  P000052188       1   1   1   1

In [72]:
w

01  02  03  04  05  06  07  08  09  10  \
year_term stage     PEOPLE_CODE_ID                                           
2018.Fall Applied   P000026021       0   0   0   0   0   0   0   0   0   0   
                    P000037722       0   0   0   0   0   0   0   0   0   0   
                    P000041297       0   0   0   0   0   0   0   0   0   0   
                    P000044821       0   0   0   0   0   0   0   0   0   0   
                    P000045054       0   0   0   0   0   0   0   0   0   0   
                    P000045209       0   0   0   0   0   1   1   1   1   1   
                    P000045865       0   0   0   0   0   0   0   0   0   0   
                    P000047104       0   0   0   0   0   0   0   0   0   0   
                    P000047242       0   0   0   0   0   0   0   0   1   1   
                    P000050259       0   0   0   0   0   0   0   0   0   0   
                    P000050922       0   0   0   0   0   0   0   0   0   0   
                    P000051053       0   0   0   0   0   0   0   0   0   0   
                    P000051145       0   0   0   0   0   0   0   0   1   1   
                    P000051235       0   0   0   0   0   0   0   0   0   0   
                    P000051293       0   0   0   0   0   1   1   1   1   1   
                    P000051681       0   0   0   0   0   0   0   0   0   0   
                    P000051745       0   0   0   0   0   0   1   1   1   1   
                    P000052188       0   0   0   0   0   0   0   0   0   0   
                    P000053208       0   0   0   0   0   0   0   0   0   0   
                    P000053326       0   0   0   0   0   0   0   0   0   0   
                    P000053398       0   0   0   1   1   1   1   1   1   1   
                    P000053572       0   0   0   0   0   0   0   0   0   0   
                    P000053576       0   0   0   0   0   0   0   1   1   1   
                    P000053577       0   0   0   0   0   0   1   1   1   1   
                    P000053580       0   0   0   0   0   0   0   1   1   1   
                    P000053595       0   0   0   0   0   0   0   0   0   0   
                    P000053870       0   0   0   0   0   0   0   0   0   0   
                    P000053874       0   0   0   0   0   0   0   0   0   0   
                    P000053942       0   0   0   0   0   0   0   0   1   1   
                    P000053976       0   0   0   0   0   0   0   0   0   0   
...                                 ..  ..  ..  ..  ..  ..  ..  ..  ..  ..   
          Deposited P000055592       0   0   0   0   0   0   0   0   0   0   
                    P000055593       0   0   0   0   0   0   0   0   0   0   
                    P000055594       0   0   0   0   0   0   0   0   0   0   
                    P000055595       0   0   0   0   0   0   0   0   0   0   
                    P000055599       0   0   0   0   0   0   0   0   0   0   
                    P000055600       0   0   0   0   0   0   0   0   0   0   
                    P000055601       0   0   0   0   0   0   0   0   0   0   
                    P000055603       0   0   0   0   0   0   0   0   0   0   
                    P000055604       0   0   0   0   0   0   0   0   0   0   
                    P000055605       0   0   0   0   0   0   0   0   0   0   
                    P000055606       0   0   0   0   0   0   0   0   0   0   
                    P000055607       0   0   0   0   0   0   0   0   0   0   
                    P000055608       0   0   0   0   0   0   0   0   0   0   
                    P000055609       0   0   0   0   0   0   0   0   0   0   
                    P000055610       0   0   0   0   0   0   0   0   0   0   
                    P000055611       0   0   0   0   0   0   0   0   0   0   
                    P000055612       0   0   0   0   0   0   0   0   0   0   
                    P000055613       0   0   0   0   0   0   0   0   0   0   
                    P000055616       0   0   0   0   0   0

In [83]:
summ = w.groupby(['year_term', 'stage']).sum()


In [74]:
summ.loc[('2018.Fall', )]

,01,02,03,04,05,06,07,08,09,10,...,44,45,46,47,48,49,50,51,52,53
stage,,,,,,,,,,,,,,,,,,,,,
Accepted,1,1,1,1,1,34,43,67,111,120,...,807,807,807,807,808,808,809,809,809,809
Applied,6,8,16,24,53,74,100,135,187,225,...,809,809,809,810,810,817,818,823,824,827
Deposited,0,0,0,0,0,0,0,2,2,2,...,145,145,145,145,145,145,145,145,145,147


In [84]:
(summ.loc[('2013.Fall', )].transpose()).to_csv('2013.Fall.csv')
(summ.loc[('2014.Fall', )].transpose()).to_csv('2014.Fall.csv')
(summ.loc[('2015.Fall', )].transpose()).to_csv('2015.Fall.csv')
(summ.loc[('2016.Fall', )].transpose()).to_csv('2016.Fall.csv')

In [85]:
(summ.loc[('2018.Fall', )].transpose()).to_csv('2018.Fall.csv')


In [87]:
writer = pd.ExcelWriter(f'admissions_historic_data_{today}.xlsx')
terms = ['2010.Fall', '2011.Fall', '2012.Fall', '2013.Fall', '2014.Fall', '2015.Fall', '2016.Fall', '2017.Fall', '2018.Fall', ]
#terms = ['2018.Fall', ]
for t in terms:
    (summ.loc[(t, )].transpose()).to_excel(writer, t)
writer.save()

In [90]:
summ_t = summ.transpose()
summ_t.head()

year_term 2009.Fall                   2009.Spring                   2010.Fall  \
stage      Accepted Applied Deposited    Accepted Applied Deposited  Accepted   
01                0       0         0           0       0         0         2   
02                0       0         0           0       0         0         3   
03                1       4         0           1       0         1         5   
04                1       4         0           1       0         1         6   
05                2       4         1           2       0         2         7   

year_term                   2010.Spring    ...    2017.Fall 2017.Spring  \
stage     Applied Deposited    Accepted    ...    Deposited    Accepted   
01              0         2           1    ...            0           0   
02              0         3           1    ...            1           0   
03              0         5           1    ...            1           0   
04              0         6           1    ...            1           2   
05              0         7           1    ...            1           2   

year_term                   2018.Fall                   2018.Spring          \
stage     Applied Deposited  Accepted Applied Deposited    Accepted Applied   
01              0         0         1       6         0           0       0   
02              0         0         1       8         0           0       1   
03              1         0         1      16         0           0       2   
04              3         0         1      24         0           0       4   
05              3         1         1      53         0           1       6   

year_term            
stage     Deposited  
01                0  
02                0  
03                0  
04                0  
05                0  

[5 rows x 60 columns]

In [92]:
summ_t.index

Index(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
       '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36',
       '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48',
       '49', '50', '51', '52', '53'],
      dtype='object')

In [93]:
summ_t[('2018.Fall', 'Deposited')]

01      0
02      0
03      0
04      0
05      0
06      0
07      0
08      2
09      2
10      2
11      2
12      2
13      3
14      3
15      3
16      3
17      3
18      3
19      4
20      6
21     18
22     30
23     44
24     53
25     55
26     75
27     85
28     92
29     98
30    103
31    105
32    117
33    126
34    136
35    145
36    145
37    145
38    145
39    145
40    145
41    145
42    145
43    145
44    145
45    145
46    145
47    145
48    145
49    145
50    145
51    145
52    145
53    147
Name: (2018.Fall, Deposited), dtype: int64

In [98]:
terms = ['2010.Fall', '2011.Fall', '2012.Fall', '2013.Fall', '2014.Fall', '2015.Fall', '2016.Fall', '2017.Fall', '2018.Fall', ]
this_term = '2018.Fall'
#print(terms)
#print(this_term)
terms.remove(this_term)

#print(terms)

['2010.Fall', '2011.Fall', '2012.Fall', '2013.Fall', '2014.Fall', '2015.Fall', '2016.Fall', '2017.Fall', '2018.Fall']
2018.Fall
['2010.Fall', '2011.Fall', '2012.Fall', '2013.Fall', '2014.Fall', '2015.Fall', '2016.Fall', '2017.Fall']


In [100]:
from bokeh.plotting import figure, output_file, show
from bokeh.palettes import Blues8


p = figure(plot_width=800, plot_height=600, title="Weekly Deposits", x_axis_label="Week Number", y_axis_label="Deposits")

c=7
for t in terms:
    p.line(summ_t.index, summ_t[(t, 'Deposited')], color=Blues8[c], legend=t)
    c -= 1

p.line(summ_t.index, summ_t[(this_term, 'Deposited')], color='red', legend=this_term)

p.legend.location = "top_left"

output_file("weekly_deposits.html")

show(p)